In [1]:
%matplotlib inline
import os
import random
import numpy as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import seaborn as sn
from sklearn import model_selection
from sklearn import preprocessing
import IPython.display as ipd
import cv2

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#!ls

In [2]:
train_file = "train.csv"
audio_dir = "audio_files/"

meta_data = pd.read_csv(train_file)
meta_data.head()

,file_name,target
0,c2c4efc4-0c7c-4696-8743-dd9d08209d71.wav,six
1,a0513500-ab86-4720-87ac-4916f06988ff.wav,five
2,7a9e9857-6a9f-4306-aab0-d2de315fae24.wav,wow
3,efc1d37d-bca9-4a84-b1cd-242396f52c73.wav,cat
4,21c3143d-3a8f-4e84-b48a-a8e25b2757c3.wav,forward


In [3]:
data_size = meta_data.shape
data_size

(74080, 2)

In [6]:
x = list(meta_data.loc[:,"file_name"])
y = list(meta_data.loc[:, "target"])

print(len(x),len(y))
target_set = set(y)
class_dic = dict()

for i,j in enumerate(target_set):
    class_dic[j] = i

print(class_dic)
x_dir = []
y_dir = []
ind = 0

while ind < len(x):
    file_path = audio_dir + x[ind]
    class_num = class_dic[y[ind]]
    x_dir.append(file_path)
    y_dir.append(class_num)

    ind = ind + 1  
    #if ind%(len(x)/10) == 0:
    #    print('process = {}%'.format(ind*10/(len(x)/10)))
    #    print("x_dir = {}, y_dir = {}".format(len(x_dir),len(y_dir)))    

x_train, x_test, y_train, y_test = model_selection.train_test_split(x_dir, y_dir, test_size=0.2, stratify=y_dir)
print("x train:{0}\ny train:{1}\nx test:{2}\ny test:{3}".format(len(x_train),
                                                                len(y_train), 
                                                                len(x_test), 
                                                                len(y_test)))
print("x_train = \n", x_train[0:10])
print("y_train = \n", y_train[0:10])                                                    

74080 74080
{'bird': 0, 'zero': 1, 'eight': 2, 'left': 3, 'backward': 4, 'four': 5, 'cat': 6, 'off': 7, 'forward': 8, 'one': 9, 'house': 10, 'dog': 11, 'tree': 12, 'go': 13, 'happy': 14, 'marvin': 15, 'visual': 16, 'no': 17, 'down': 18, 'bed': 19, 'up': 20, 'yes': 21, 'sheila': 22, 'learn': 23, 'follow': 24, 'right': 25, 'stop': 26, 'three': 27, 'six': 28, 'on': 29, 'five': 30, 'seven': 31, 'nine': 32, 'wow': 33, 'two': 34}
x train:59264
y train:59264
x test:14816
y test:14816
x_train = 
 ['audio_files/47c38c0c-6d2b-4c55-8acc-dfa3b20bc0c6.wav', 'audio_files/dd2e3aa1-b3b5-4006-ae8d-ed15be7378b7.wav', 'audio_files/29ce847b-742b-47fa-a96d-4730f1d86d08.wav', 'audio_files/0b18a604-9097-4c48-9fc9-c1273ae7e1e6.wav', 'audio_files/59cc42f7-fd45-4a7c-a892-d162407ace67.wav', 'audio_files/30847870-eb87-4e88-8f0f-cee678250d6a.wav', 'audio_files/91c3b48b-a65d-4323-b0d6-545c2dfd904e.wav', 'audio_files/5c590ac0-1578-48da-8479-9554545c0dbc.wav', 'audio_files/ca6b84f0-de91-4bef-bd17-4d4d3f05f41b.wav', '

In [7]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import torch.nn.functional as F
import torchvision
from torch.autograd import Variable

from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [8]:
class SpeechDataset(Dataset):
    def __init__(self, data_path, target=None, is_test=False, augmentation=None):
        super().__init__()
        self.data_path = data_path
        self.target = target
        self.is_test = is_test
        self.augmentation = augmentation
        self.duration = 1000
        self.sr = 16000
        self.n_fft = 1024
        self.hop_length = None
        self.n_mels = 64
        self.top_db = 80

    def __len__(self):
        return len(self.data_path)

    # def __getitem__(self, item):
    #     wav, sr = load_wave_data(self.data_path[item])
    #     #melsp_x = librosa.feature.melspectrogram(y=wav, sr=sr, hop_length=512, n_mels=32, n_fft=2048)
    #     #melsp_x = librosa.feature.melspectrogram(y=norm_wav, sr=16000, hop_length=512, n_mels=32, n_fft=2048)
    #     #melsp_x = calculate_melsp(wav,n_mels=64,n_fft=1024,hop_length=256)
    #     melsp_x = calculate_melsp(wav,n_mels=64,n_fft=1024,hop_length=None)
    #     norm_melsp = (melsp_x - sample_mean) / (sample_std + 1e-6)

    #     norm_melsp = norm_melsp[np.newaxis, :, :]
    #     norm_melsp = norm_melsp.astype('float32')
    #     norm_melsp = torch.from_numpy(norm_melsp)
    #     return norm_melsp, self.target[item]

    def __getitem__(self,idx):
        file_path = self.data_path[idx]
        class_id = self.target[idx]

        samples,sr = torchaudio.load(file_path)
        samples = self._pad_trunc(samples,self.sr)

        spect = torchaudio.transforms.MelSpectrogram(
            self.sr,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            n_mels=self.n_mels
        )(samples)

        spect = torchaudio.transforms.AmplitudeToDB(top_db=self.top_db)(spect)
        spect = self.rechannel(spect,self.sr,3)

        return spect, self.target[idx]
        

    def _pad_trunc(self, samples, sr):
        num_rows, signal_len = samples.shape
        max_len = sr // 1000 * self.duration

        if (signal_len > max_len):
            # Truncate the signal to the given length
            samples = samples[:, max_len]

        elif (signal_len < max_len):
            # Length of padding to add at the beginning and end of the signal
            pad_begin_len = random.randint(0, max_len - signal_len)
            pad_end_len = max_len - signal_len - pad_begin_len

            # Pad with 0s
            pad_begin = torch.zeros((1, pad_begin_len))
            pad_end = torch.zeros((1, pad_end_len))

            samples = torch.cat((pad_begin, samples, pad_end), 1)

        return samples

    def rechannel(self, spect, sr, num_channel):
        if (spect.shape[0] == num_channel):
            # Nothing to do
            return spect

        if (num_channel == 1):
            # Convert from stereo to mono by selecting only the first channel
            spect = spect[:1, :]
        else:
            # Convert from mono to stereo by duplicating the first channel
            spect = torch.cat([spect, spect, spect])

        return spect

    def _time_shift(self, samples, sr, shift_limit):
        _, sig_len = samples.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return samples.roll(shift_amt)
    


In [9]:
config = {
    'batch_size': 64,
    'num_workers': 0,
    'epochs': 10,
    'device': 'cpu'
}


In [10]:
train_dataset = SpeechDataset(
    data_path = x_train,
    target = y_train,
    is_test = False
)
valid_dataset = SpeechDataset(
    data_path = x_test,
    target = y_test,
    is_test = False
)
train_loader = DataLoader(
    train_dataset,
    batch_size = config['batch_size'],
    shuffle = True,
    num_workers = config['num_workers'],
    drop_last = True
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size = config['batch_size'],
    shuffle = False,
    num_workers = config['num_workers'],
    drop_last = False
)

In [12]:
model = torchvision.models.resnet18(num_classes=35)
#model = torchvision.models.mobilenet_v3_small(num_classes=35)
#model.features[0][0] = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
model.to(config['device'])

# 获取优化方法
optimizer = torch.optim.Adam(params=model.parameters(), lr=1e-4, betas=(0.9,0.99), eps=1e-6, weight_decay=5e-4)
#optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-2)
# 获取学习率衰减函数
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)
#scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[1, 2, 3, 4], gamma=0.3)
# 获取损失函数
loss = torch.nn.CrossEntropyLoss()
device = 'cpu'

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [18]:
def validation(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            
    test_loss /= num_batches
    correct /= size
    print(f"Val Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    

In [22]:
epochs = 30
for t in tqdm(range(epochs)):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_loader, model, loss, optimizer)
    validation(valid_loader, model, loss)

print("Done!")


  0%|          | 0/30 [00:00<?, ?it/s]

Epoch 1
-------------------------------
loss: 2.481280  [    0/59264]
loss: 1.703591  [ 6400/59264]
loss: 1.234425  [12800/59264]
loss: 1.049305  [19200/59264]
loss: 0.965315  [25600/59264]
loss: 0.907588  [32000/59264]
loss: 0.742032  [38400/59264]
loss: 0.751323  [44800/59264]
loss: 0.496773  [51200/59264]
loss: 0.571004  [57600/59264]


  3%|▎         | 1/30 [07:39<3:42:02, 459.38s/it]

Val Error: 
 Accuracy: 81.0%, Avg loss: 0.625335 

Epoch 2
-------------------------------
loss: 0.727611  [    0/59264]
loss: 0.672994  [ 6400/59264]
loss: 0.525397  [12800/59264]
loss: 0.631269  [19200/59264]
loss: 0.359109  [25600/59264]
loss: 0.356472  [32000/59264]
loss: 0.567875  [38400/59264]
loss: 0.430809  [44800/59264]
loss: 0.392609  [51200/59264]
loss: 0.377854  [57600/59264]


  7%|▋         | 2/30 [15:16<3:33:46, 458.10s/it]

...

Val Error: 
 Accuracy: 91.3%, Avg loss: 0.347277 

Epoch 29
-------------------------------
loss: 0.009500  [    0/59264]
loss: 0.008876  [ 6400/59264]
loss: 0.051380  [12800/59264]
loss: 0.037584  [19200/59264]
loss: 0.111212  [25600/59264]
loss: 0.023766  [32000/59264]
loss: 0.122791  [38400/59264]
loss: 0.078754  [44800/59264]
loss: 0.062445  [51200/59264]
loss: 0.069912  [57600/59264]


 97%|█████████▋| 29/30 [3:41:08<07:35, 455.75s/it]

Val Error: 
 Accuracy: 90.5%, Avg loss: 0.377131 

Epoch 30
-------------------------------
loss: 0.012395  [    0/59264]
loss: 0.016510  [ 6400/59264]
loss: 0.022604  [12800/59264]
loss: 0.115674  [19200/59264]
loss: 0.018079  [25600/59264]
loss: 0.086660  [32000/59264]
loss: 0.064706  [38400/59264]
loss: 0.036662  [44800/59264]
loss: 0.031165  [51200/59264]
loss: 0.022986  [57600/59264]


100%|██████████| 30/30 [3:48:43<00:00, 457.45s/it]

Val Error: 
 Accuracy: 91.2%, Avg loss: 0.356323 

Done!


In [23]:
submission = pd.read_csv("sample_submission.csv")
test_filename = list(submission.loc[:,"file_name"])
test_dir = []
for item in test_filename:
  ans = audio_dir + item
  test_dir.append(ans)

test_dir[0:10]

['audio_files/6f757c30-32f8-4857-a4e6-53a8b154b838.wav',
 'audio_files/36c4e1b3-9427-4905-8653-1e43bbddd938.wav',
 'audio_files/59599150-4db7-494b-8b8a-d35eee129b70.wav',
 'audio_files/a71314a4-afcf-465e-9f17-0d5768b69dce.wav',
 'audio_files/cc61243b-a830-4af7-94f2-962787e5268f.wav',
 'audio_files/0cc0aaa2-8a2e-4fff-9f2a-48c3b8736d48.wav',
 'audio_files/fce534da-f644-435c-855a-77512872dcd1.wav',
 'audio_files/0c8d2f3e-8538-4118-a096-7ddcdeddc331.wav',
 'audio_files/b0049719-427a-409d-848a-8c10b88a9799.wav',
 'audio_files/cb6be4cc-f462-44bd-ac99-784a9ae4c811.wav']

In [27]:
def pad_trunc(samples, sr):
    num_rows, signal_len = samples.shape
    max_len = sr // 1000 * 1000

    if (signal_len > max_len):
        # Truncate the signal to the given length
        samples = samples[:, max_len]

    elif (signal_len < max_len):
        # Length of padding to add at the beginning and end of the signal
        pad_begin_len = random.randint(0, max_len - signal_len)
        pad_end_len = max_len - signal_len - pad_begin_len

        # Pad with 0s
        pad_begin = torch.zeros((1, pad_begin_len))
        pad_end = torch.zeros((1, pad_end_len))

        samples = torch.cat((pad_begin, samples, pad_end), 1)

    return samples

def rechannel(spect, sr, num_channel):
        if (spect.shape[0] == num_channel):
            # Nothing to do
            return spect

        if (num_channel == 1):
            # Convert from stereo to mono by selecting only the first channel
            spect = spect[:1, :]
        else:
            # Convert from mono to stereo by duplicating the first channel
            spect = torch.cat([spect, spect, spect])

        return spect

def time_shift(samples, sr, shift_limit):
        _, sig_len = samples.shape
        shift_amt = int(random.random() * shift_limit * sig_len)
        return samples.roll(shift_amt)

def get_keys(d, value):
    return [k for k,v in d.items() if v == value]

result = []
for fpath in tqdm(test_dir):   
    samples, sr = torchaudio.load(fpath)
    samples = pad_trunc(samples,16000)

    spect = torchaudio.transforms.MelSpectrogram(
        sample_rate = 16000,
        n_fft=1024,
        hop_length=None,
        n_mels=64
    )(samples)

    spect = torchaudio.transforms.AmplitudeToDB(top_db=80)(spect)
    spect = rechannel(spect,16000,3)
    spect = spect[np.newaxis,:]

    
    output = model(spect)
    ans = torch.nn.functional.softmax(output)
    ans = ans.data.cpu().numpy()
    lab = np.argsort(ans)[0][-1]
    result.append(lab)
    
res1 = []
for elem in result:
    tg = get_keys(class_dic, elem)
    res1.append(tg[0])    

submission["target"] = res1
submission.to_csv("new_submission.csv", index=None)
submission.head()

  0%|          | 0/31749 [00:00<?, ?it/s]/var/folders/j8/287qv_h962sgjzzwzh6whkpm0000gn/T/ipykernel_13937/2614235750.py:70: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ans = torch.nn.functional.softmax(output)
100%|██████████| 31749/31749 [03:23<00:00, 156.32it/s]


,file_name,target
0,6f757c30-32f8-4857-a4e6-53a8b154b838.wav,follow
1,36c4e1b3-9427-4905-8653-1e43bbddd938.wav,four
2,59599150-4db7-494b-8b8a-d35eee129b70.wav,bird
3,a71314a4-afcf-465e-9f17-0d5768b69dce.wav,no
4,cc61243b-a830-4af7-94f2-962787e5268f.wav,left
